In [2]:
import datetime
from pycti import OpenCTIApiClient
from dateutil.parser import parse
import pandas as pd
import os
import warnings

In [23]:
api_url = "https://demo.opencti.io"
api_token = "2b4f29e3-5ea8-4890-8cf5-a76f61f1e2b2"

# OpenCTI initialization
opencti_api_client = OpenCTIApiClient(api_url, api_token)

INFO:root:Listing Threat-Actors with filters null.


In [51]:
opencti_api_client.stix_observable.list(search='fc236f15-18fe-4a85-a75f-9f684afe0afb')

INFO:root:Listing StixObservables with filters null.


[{'id': 'fc236f15-18fe-4a85-a75f-9f684afe0afb',
  'stix_id_key': 'observable--4f914ba3-fbc8-41f6-8bfa-0c5c4dd69caf',
  'entity_type': 'ipv4-addr',
  'name': '66.79.174.219',
  'description': 'IP source of a malicious email',
  'observable_value': '66.79.174.219',
  'created_at': '2020-05-05T20:29:29.993Z',
  'updated_at': '2020-05-05T20:29:29.993Z',
  'createdByRef': {'id': 'aa649146-63bd-4bd5-b31c-f5369187dd4b',
   'entity_type': 'organization',
   'stix_id_key': 'identity--6ee3c152-04d7-418d-9551-8c6a691bbc72',
   'stix_label': None,
   'name': 'CthulhuSPRL.be',
   'alias': [],
   'description': '',
   'created': '2020-04-30T07:47:52.878Z',
   'modified': '2020-04-30T07:47:52.878Z',
   'organization_class': None,
   'remote_relation_id': 'abe3e45d-e977-55b0-a361-8b4d3883d538'},
  'tags': [],
  'markingDefinitions': [{'id': '1e0f3cce-c736-4d89-8aa5-9dbfbccf98dd',
    'entity_type': 'marking-definition',
    'stix_id_key': 'marking-definition--34098fce-860f-48ae-8e50-ebd3cc5e41da',
   

In [49]:
observable_ttp1 = opencti_api_client.stix_observable.create(type="Email-Address", observable_value="phishing@mail.com", createIndicator=True)

INFO:root:Listing StixObservables with filters [{"key": "observable_value", "values": ["phishing@mail.com"]}].


In [57]:
observable_ttp1 = opencti_api_client.stix_observable.create(type="IPv4-Addr", observable_value='58.246.0.0', createIndicator=True)

INFO:root:Listing StixObservables with filters [{"key": "observable_value", "values": ["58.246.0.0"]}].
INFO:root:Creating Stix-Observable {58.246.0.0} with indicator at True.


In [3]:
def threat_group_details(threat_group_name,report_id):
    intrusion_set = opencti_api_client.intrusion_set.read(filters=[{"key": "name", "values": [threat_group_name]}])
    stix_relations = opencti_api_client.stix_relation.list(fromId=intrusion_set["id"], inferred=True)
    all_details=[]
    for relation in stix_relations:
        details=[relation['from']['id'],relation['id'],relation['to']['id'],relation['from']['name'],relation['relationship_type'],relation['to']['name'],relation['to']['entity_type']]
        all_details.append(details)
    df_relations=pd.DataFrame(all_details,columns=['from_id','relation_id','to_id','from_name','relation','to_name','to_type'])
    
    all_id=list(set(df_relations['from_id']))
    all_id.extend(set(df_relations['to_id']))
    all_id.extend(set(df_relations['relation_id']))
    
    result =list(map(lambda x:opencti_api_client.report.add_stix_entity(id='3074f086-d500-410b-b7c9-86a4bb17b12d',entity_id =x),all_id))
    
    return df_relations

In [4]:
df_relations=threat_group_details('Carbanak')

INFO:root:Listing Intrusion-Sets with filters [{"key": "name", "values": ["Carbanak"]}].
INFO:root:Listing stix_relations with {type: None, from_id: 31439558-cf5c-4827-b60f-bca3da19a426, to_id: None}


In [5]:
df_relations

,from_id,relation_id,to_id,from_name,relation,to_name,to_type
0,31439558-cf5c-4827-b60f-bca3da19a426,ad8a242e-f404-4873-a001-4443ca915d9c,f74bff8b-7621-4813-94b5-c317c0bd5358,Carbanak,uses,Masquerading,attack-pattern
1,31439558-cf5c-4827-b60f-bca3da19a426,de330995-63e0-4223-a780-679f2c4fa2ca,02286470-03b4-4265-b13b-3b73f5a77065,Carbanak,uses,Valid Accounts,attack-pattern
2,31439558-cf5c-4827-b60f-bca3da19a426,779d8dc5-5c63-435f-8e43-b949340cb522,a2286881-bcaa-4c9c-8960-246169b8b3d0,Carbanak,uses,Mimikatz,tool
3,31439558-cf5c-4827-b60f-bca3da19a426,64ff87a5-1067-402e-9402-0ac487c555dd,de3f83d3-4488-4a63-b64c-46e8cb7559c2,Carbanak,uses,New Service,attack-pattern
4,31439558-cf5c-4827-b60f-bca3da19a426,c57f5f2f-2d2e-4d68-9d3e-4822aea02899,3d5acbd6-257e-4e52-903a-b692dcfe29dd,Carbanak,uses,Carbanak,malware
5,31439558-cf5c-4827-b60f-bca3da19a426,fde98d27-63db-45bf-8c2a-35f4eaa377a6,d2f203f9-5d49-42ba-a27d-82108676b9b7,Carbanak,uses,netsh,tool
6,31439558-cf5c-4827-b60f-bca3da19a426,7cdfa2b9-4eab-408b-a0ba-e56c800dbd63,1240e186-1cd9-425e-bdaf-ce71a2d9419a,Carbanak,uses,Web Service,attack-pattern


In [67]:
a=list(set(df_relations['from_id']))
a.extend(set(df_relations['to_id']))
a.extend(set(df_relations['relation_id']))

In [72]:
result =list(map(lambda x:opencti_api_client.report.add_stix_entity(id='3074f086-d500-410b-b7c9-86a4bb17b12d',entity_id =x),a))

INFO:root:Checking Stix-Entity {93dd15f3-e4be-4bde-ab5a-ec74ceb467b6} in Report {3074f086-d500-410b-b7c9-86a4bb17b12d}
INFO:root:Adding Stix-Entity {93dd15f3-e4be-4bde-ab5a-ec74ceb467b6} to Report {3074f086-d500-410b-b7c9-86a4bb17b12d}
INFO:root:Checking Stix-Entity {fbf93253-1bac-4f23-88e1-d91e35e357a0} in Report {3074f086-d500-410b-b7c9-86a4bb17b12d}
INFO:root:Adding Stix-Entity {fbf93253-1bac-4f23-88e1-d91e35e357a0} to Report {3074f086-d500-410b-b7c9-86a4bb17b12d}
INFO:root:Checking Stix-Entity {fe57226d-e1a0-441c-9c7e-f6fa252b6d13} in Report {3074f086-d500-410b-b7c9-86a4bb17b12d}
INFO:root:Adding Stix-Entity {fe57226d-e1a0-441c-9c7e-f6fa252b6d13} to Report {3074f086-d500-410b-b7c9-86a4bb17b12d}
INFO:root:Checking Stix-Entity {52e3b617-23e3-477a-a38e-9174e895a071} in Report {3074f086-d500-410b-b7c9-86a4bb17b12d}
INFO:root:Adding Stix-Entity {52e3b617-23e3-477a-a38e-9174e895a071} to Report {3074f086-d500-410b-b7c9-86a4bb17b12d}
INFO:root:Checking Stix-Entity {92d06189-7d26-4f53-99f6-

INFO:root:Checking Stix-Entity {f16ec627-9c46-4f97-8fc2-e3539570a0fa} in Report {3074f086-d500-410b-b7c9-86a4bb17b12d}
INFO:root:Adding Stix-Entity {f16ec627-9c46-4f97-8fc2-e3539570a0fa} to Report {3074f086-d500-410b-b7c9-86a4bb17b12d}
INFO:root:Checking Stix-Entity {556e5140-5054-4950-a2f8-392303d0d903} in Report {3074f086-d500-410b-b7c9-86a4bb17b12d}
INFO:root:Adding Stix-Entity {556e5140-5054-4950-a2f8-392303d0d903} to Report {3074f086-d500-410b-b7c9-86a4bb17b12d}
INFO:root:Checking Stix-Entity {89f4f431-d956-4db6-b02a-87c7eea968f5} in Report {3074f086-d500-410b-b7c9-86a4bb17b12d}
INFO:root:Adding Stix-Entity {89f4f431-d956-4db6-b02a-87c7eea968f5} to Report {3074f086-d500-410b-b7c9-86a4bb17b12d}
INFO:root:Checking Stix-Entity {15c1d1c9-31ed-4fc5-a07d-ca1207190bff} in Report {3074f086-d500-410b-b7c9-86a4bb17b12d}
INFO:root:Adding Stix-Entity {15c1d1c9-31ed-4fc5-a07d-ca1207190bff} to Report {3074f086-d500-410b-b7c9-86a4bb17b12d}
INFO:root:Checking Stix-Entity {eb45747d-91fe-4f93-a05d-

In [70]:
l = ['sat', 'bat', 'cat', 'mat'] 
  
# map() can listify the list of strings individually 
test = map(list, l)
print(test) 

In [71]:
numbers1 = [1, 2, 3] 
numbers2 = [4, 5, 6] 
  
result = map(lambda x, y: x + y, numbers1, numbers2) 
print(list(result))

[5, 7, 9]
